# maps

#### This notebook includes several maps that visualize data (possibly) related to gun violence in the US. Sources for data will be listed here as well as the README file in the parent directory. To do this analysis, I'm using the following libraries:


In [1]:
import folium as fm
import pandas as pd
import numpy as np

In [2]:
# This is a dataset from motherjones.com compiling info on mass shootings.
# For more info, see README.
mj_shootings = pd.read_csv('mj_1982-2016.csv')

In [3]:
mj_shootings.columns

Index(['Case', 'Location', 'Date', 'Year', 'Summary', 'Fatalities', 'Wounded',
       'Total victims', 'Venue', 'Prior signs of possible mental illness',
       'Mental Health', 'Weapons obtained legally', 'Where obtained',
       'Type of weapons', 'Weapon details', 'Race', 'Gender', 'Sources',
       'Mental Health Sources', 'latitude', 'longitude', 'Type'],
      dtype='object')

In [4]:
# Where have these shootings taken place? Let's load the data into a folium map.
# Since we'll be resetting the map fairly often, let's define this as a method

def get_us_shootings():
    us_map = fm.Map(location=[48, -102], zoom_start=3)
    
    i = 0
    while (i < len(mj_shootings.index)):
        case = mj_shootings.iloc[i].Case
        lat = mj_shootings.iloc[i].latitude
        long = mj_shootings.iloc[i].longitude
        victims = mj_shootings.iloc[i]['Total victims']
        
        if not np.isnan(lat):
            if not np.isnan(long):
                fm.CircleMarker([lat, long], radius=victims*1000, popup=case,
                                 color='#f61111', fill_color='#f61111').add_to(us_map)
        
        i+=1
    
    return us_map

In [5]:
get_us_shootings()

In [6]:
# Let's pull in information about which types of guns were used, and what the laws are for each state.
long_gun_laws = pd.read_csv('state_long_gun_laws.csv')
handgun_laws = pd.read_csv('state_handgun_laws.csv')

long_gun_laws.head()

,state,permit_to_purchase,firearm_registration,license_to_own,ccw_permit_issued,open_carry,state_preemption,assault_weapon_law,magazine_restrictions,nfa_restrictions,...,stand_your_ground,self_defense_law,duty_to_retreat,vehicle_carry,unlicensed_open_carry,concealed_carry_college,waiting_period,opt_out_statute,out_of_state_recognized,right_to_bear_arms
0,Alabama,No,No,No,No,Yes,Yes,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,No,No,No,No,Yes,Yes,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arizona,No,No,No,Yes,Yes,Partial,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arkansas,No,No,No,No,Yes,Yes,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,California,Partial,No,No,Yes,Partial,Yes,Yes,Yes,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN


In [7]:
# This data is going to be hard to manipulate as a set of strings. Let's define some numeric values as placeholders:
# No = -1
# Yes = 1
# {any other text} = 0

lg = long_gun_laws.copy()
lg = lg.fillna(0)
lg = lg.replace('Yes', 1)
lg = lg.replace('No', -1)
lg = lg.replace('Partial', 0)

lg.head()

,state,permit_to_purchase,firearm_registration,license_to_own,ccw_permit_issued,open_carry,state_preemption,assault_weapon_law,magazine_restrictions,nfa_restrictions,...,stand_your_ground,self_defense_law,duty_to_retreat,vehicle_carry,unlicensed_open_carry,concealed_carry_college,waiting_period,opt_out_statute,out_of_state_recognized,right_to_bear_arms
0,Alabama,-1,-1,-1,-1,1,1,-1,0,-1,...,0,0,0,0,0,0,0,0,0,0
1,Alaska,-1,-1,-1,-1,1,1,-1,0,-1,...,0,0,0,0,0,0,0,0,0,0
2,Arizona,-1,-1,-1,1,1,0,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
3,Arkansas,-1,-1,-1,-1,1,1,-1,-1,0,...,0,0,0,0,0,0,0,0,0,0
4,California,0,-1,-1,1,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0


In [8]:
gun_law_map = get_us_shootings()

gun_law_map.choropleth(geo_path=r'us_states_2010.json',
                       data=lg, 
                       columns=['state', 'permit_to_purchase'],
                       key_on='feature.properties.NAME',
                       fill_color='BuPu',
                       fill_opacity=0.3,
                       line_opacity=0.2)

gun_law_map

#TODO: pull the keys from the json file into the dataset so folium won't throw a KeyError.
# see: http://stackoverflow.com/questions/37003421/folium-choropleth-geojson-raises-attributeerror-nonetype

c:\users\localadmin\miniconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


KeyError: 'Puerto Rico'